In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import math
from gensim.models import KeyedVectors
from tqdm.notebook import tqdm
os.chdir('/gdrive/My Drive/NLP_Project_Programs/Paper_Implementation')
#os.listdir()

In [0]:
path = 'Evaluation_Benchmarks/Syntactic_Relns/syntactic_alone.txt'  # path to Syntactic Relations evaluation dataset file
obj = open(path,'r')
syn_words = []
for i,line in enumerate(obj):
  syn_words.append(line.strip().lower().split())
# REDUCE THE VECTOR SPACE OF SKIPGRAM BY TAKING ONLY THOSE WORDS IN SYN-REL DATASET AND DISCARD THE REST.
# THIS WILL SPEED UP THE COSINE SIMILARITY CALCULATION (HOPEFULLY).

# Obtain all the unique words
unique_words = []
for line in syn_words:
    for word in line:
        unique_words.append(word)
unique_words = set(unique_words)

def reduce_vectors(words, vectors):
    new_words = []
    new_vectors = []
    
    for word, vector in zip(words, vectors):
        if word in unique_words:
            new_words.append(word)
            new_vectors.append(vector)

    unique_reduced_words = set(new_words)
    missing_words = unique_words - unique_words.intersection(unique_reduced_words)
    for word, vector in tqdm(zip(words, vectors)):
        word = word.lower()
        if word in missing_words:
            new_words.append(word)
            new_vectors.append(vector)
    return new_words, new_vectors

def SynRel_model(words, vectors, vector_dimensions):
  """
  Computes accuracy for the task of selecting the word closest analogous word d such that a:b::c:d using cosine similarity

  Parameters
  ----------
  words : list
    A list containing only words (dtype = string) from given word vector file

  vectors : list
    A list of lists with each sub-list as vector (dtype = float) for corresponding word from given word vector file  

  vector_dimensions : int 
    The length of the vectors present in given word-vector file

  Returns
  -------
  arg1 : float
    accuracy score for the evaluation task
  """
  count = 0
  missed_count=0
  accuracy = 0
  vec_mat = np.array(vectors)
  #mean_vector = np.mean(vec_mat, axis=0)

  for index,wlist in tqdm(enumerate(syn_words)):
    #if index%1000==0: print('-'*15, index,'/ 10,000, \t Score :', count*100/(index+1), count*100/(index-missed_count+1))
    w4 = wlist[3]
    w_mat = np.zeros((3,vector_dimensions))
    # for i in range(3):
    #     w_mat[3] == mean_vector
    query_indices = []
    query_index = 0
    for word,vec in zip(words,vectors):
      if wlist[0] == word:
        w_mat[0,:] = vec 
        query_indices.append(query_index)
      if wlist[1] == word:
        w_mat[1,:] = vec
        query_indices.append(query_index)
      if wlist[2] == word: 
        w_mat[2,:] = vec
        query_indices.append(query_index)
      query_index+=1
    # print(query_indices)
    # break
    temp_bool = w_mat == 0
    if np.sum(temp_bool[0,:])==300:
      missed_count+=1
    if np.sum(temp_bool[1,:])==300:
      missed_count+=1
    if np.sum(temp_bool[2,:])==300:
      missed_count+=1

    q_vec = w_mat[1,:]-w_mat[0,:]+w_mat[2,:]
    q_vec = q_vec.reshape(1,vector_dimensions)

    '''--------------------- FOR DEBUGGING ------------------------'''
    # top_2_ind = np.argsort(cosine_similarity(q_vec, vec_mat))[0,-2:]
    #q_list.append(words[ind])
    # if words[top_2_ind[1]] == wlist[2]:
    #   if words[top_2_ind[0]] == w4:
    #     count += 1
    # elif words[top_2_ind[1]] == w4:
    #   count += 1
    # if (words[top_2_ind[0]] == w4) or (words[top_2_ind[1]] == w4):
    #     count += 1

    vec_without_query = np.delete(vec_mat, query_indices,0)
    words_without_query = np.delete(words, query_indices,0)
    # print('new vec shape :', np.shape(vec_without_query), 'Old shape :', np.shape(vec_mat))
    # break
    top_2_ind = np.argsort(cosine_similarity(q_vec, vec_without_query))[0,-2:]
    # if (words[top_ind[0]] == w4) or (words[top_ind[1]] == w4):
    if words_without_query[top_2_ind[1]] == wlist[2]:
      if words_without_query[top_2_ind[0]] == w4:
        count += 1
    elif words_without_query[top_2_ind[1]] == w4:
      count += 1

  print('->'*15, 'Total Missed :', missed_count)
  accuracy = count/len(syn_words)

  return accuracy

In [0]:
#Global Context word vectors file
WordVec_path_a = 'Vector Representation of Words/3/3_word_vectors.txt'
#Skip Gram (SG) word vectors file
WordVec_path_b = 'Vector Representation of Words/2/GoogleNews-vectors-negative300.bin'
#Glove word vector file
WordVec_path_c = 'Vector Representation of Words/1/glove.6B.300d.txt'

#Global Context retrofitted word vectors file
outfile_path_a1 = 'Vector Representation of Words/3/RETRO_FrameNet_3_word_vectors.txt' 
outfile_path_a2 = 'Vector Representation of Words/3/RETRO_ppdb-xl_3_word_vectors.txt' 
outfile_path_a3 = 'Vector Representation of Words/3/RETRO_wordnet-synonyms_3_word_vectors.txt' 
outfile_path_a4 = 'Vector Representation of Words/3/RETRO_wordnet-synonyms+_3_word_vectors.txt' 

#Glove retrofitted word vector files
outfile_path_c1 = 'Vector Representation of Words/1/RETRO_FrameNet_glove.6B.300d.txt'
outfile_path_c2 = 'Vector Representation of Words/1/RETRO_ppdb-xl_glove.6B.300d.txt'
outfile_path_c3 = 'Vector Representation of Words/1/RETRO_wordnet-synonyms_glove.6B.300d.txt'
outfile_path_c4 = 'Vector Representation of Words/1/RETRO_wordnet-synonyms+_glove.6B.300d.txt'


In [0]:
'''Evaluating on the Glove and GC word vectors file (including retrofitted Glove and GC word vectors)'''

''' Read all the word vectors and normalize them '''
def read_word_vecs(filename):
  # Use this for Glove and Global Context Vectors
  words = []
  vectors = []
  fileObject = open(filename, 'r')
  for index,line in enumerate(fileObject):
    # if index%50000==0:print('->', index,'/ 3000000')
    line = line.strip().lower()
    words.append(line.split()[0])
    vec = np.zeros(len(line.split())-1, dtype=float)
    for ind, vecVal in enumerate(line.split()[1:]):
      vec[ind] = float(vecVal)
    ''' normalize weight vector '''
    # vec /= math.sqrt((vec**2).sum() + 1e-6)
    vectors.append(vec)
    
  return words, vectors

''' change 2nd argument inside tuple to 50 or 300 accordingly for Global context (50) and Glove word vector files respectively '''   
filenames = [(outfile_path_a4,50), (outfile_path_c4,300)]

for filename in filenames:
  words,vectors = read_word_vecs(filename[0])
  words, vectors = reduce_vectors(words,vectors) 
  print('word vec generated for :', filename)
  # accuracy_score= SynRel_model(words,vectors,vector_dimensions=filename[1])
  accuracy_score = SynRel_model(words,vectors,vector_dimensions=filename[1])
  print('Accuracy :',accuracy_score)

In [0]:
'''Evaluating on the SG (skip gram) word vectors file (does not include retrofitted SG word vectors)'''

filename = WordVec_path_b 
def read_word_vectors_skipgram():
  # Use this for Google skipgram word2vec model
  word_vec_bin = KeyedVectors.load_word2vec_format(filename, binary=True)
  return word_vec_bin

## function to convert word_vec to (words, vectors)
def my_function(word_vec):
  words = []
  vectors = []
  vocabulary = word_vec.vocab
  for word in tqdm(vocabulary):
      words.append(word)  
      vectors.append(word_vec[word])
  
  return words, vectors    

word_vec = read_word_vectors_skipgram()
words, vectors = my_function(word_vec)
words, vectors = reduce_vectors(words,vectors)
print('word vec generated for :', filename)
accuracy_score= SynRel_model(words,vectors,vector_dimensions=filename[1])
print('Accuracy :',accuracy_score)
